Building a question classifier using neural
networks

In [1]:
#Importing basic libraries
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
train_data=open('/content/training_data.txt','r+')
test_data=open('/content/test_dataset.txt','r+')

In [3]:
train = pd.DataFrame(train_data.readlines(), columns =['Question'])
test = pd.DataFrame(test_data.readlines(), columns = ['Question'])


In [4]:
train.head()

,Question
0,DESC:manner How did serfdom develop in and the...
1,ENTY:cremat What films featured the character ...
2,DESC:manner How can I find a list of celebriti...
3,ENTY:animal What fowl grabs the spotlight afte...
4,ABBR:exp What is the full form of .com ?\n


Our dataset offers a unique challenge in terms of segregating it into Questions
and Question Types, which are attached together. Furthermore, as the Question
Type consists of both coarse and fine classes, we will need to perform separation
for these as well.

In [5]:
train['QType'] = train.Question.apply(lambda x: x.split(' ', 1)[0])
train['Question'] = train.Question.apply(lambda x: x.split(' ',1)[1])
train['QType-Coarse'] = train.QType.apply(lambda x:x.split(':')[0])
train['QType-Fine'] = train.QType.apply(lambda x: x.split(':')[1])
test['QType'] = test.Question.apply(lambda x: x.split(' ', 1)[0])
test['Question'] = test.Question.apply(lambda x: x.split(' ',1)[1])
test['QType-Coarse'] = test.QType.apply(lambda x: x.split(':')[0])
test['QType-Fine'] = test.QType.apply(lambda x: x.split(':')[1])


In [6]:
train.head()

,Question,QType,QType-Coarse,QType-Fine
0,How did serfdom develop in and then leave Russ...,DESC:manner,DESC,manner
1,What films featured the character Popeye Doyle...,ENTY:cremat,ENTY,cremat
2,How can I find a list of celebrities ' real na...,DESC:manner,DESC,manner
3,What fowl grabs the spotlight after the Chines...,ENTY:animal,ENTY,animal
4,What is the full form of .com ?\n,ABBR:exp,ABBR,exp


In [7]:
#removing other column as we are concened only with Coarse
train.pop('QType')
train.pop('QType-Fine')
test.pop('QType')
test.pop('QType-Fine')

0           dist
1           city
2           desc
3            def
4           date
         ...    
495          ind
496     currency
497        count
498    substance
499          def
Name: QType-Fine, Length: 500, dtype: object

In [8]:
train.head()

,Question,QType-Coarse
0,How did serfdom develop in and then leave Russ...,DESC
1,What films featured the character Popeye Doyle...,ENTY
2,How can I find a list of celebrities ' real na...,DESC
3,What fowl grabs the spotlight after the Chines...,ENTY
4,What is the full form of .com ?\n,ABBR


In [9]:
classes = np.unique(np.array(train['QType-Coarse']))
classes

array(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype=object)

Converting labels into vectors

In [10]:
le = LabelEncoder()
le.fit(pd.Series(train['QType-Coarse'].tolist() + test['QType-Coarse'].tolist()).values)
train['QType-Coarse'] = le.transform(train['QType-Coarse'].values)
test['QType-Coarse'] = le.transform(test['QType-Coarse'].values)


Using preprocessing pipelines

In [11]:
all_corpus = pd.Series(train.Question.tolist() + test.Question.tolist()).astype(str)


In [12]:
def text_clean(corpus):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)

    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process

    Output : Returns the cleaned text corpus

    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

In [13]:

def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [14]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [15]:

def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [16]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):

    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)

    Input :
    'corpus' - Text corpus on which pre-processing tasks will be performed

    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer

    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together

    Output : Returns the processed text corpus

    '''
    if cleaning == True:
        corpus = text_clean(corpus)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]


    return corpus

In [17]:

all_corpus = preprocess(all_corpus, remove_stopwords = True)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-fa22659f3d2e>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-12-fa22659f3d2e>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-12-fa22659f3d2e>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-12-fa22659f3d2e>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipyt

In [18]:
train_corpus = all_corpus[0:train.shape[0]]
test_corpus = all_corpus[train.shape[0]:]


TF-IDF based Vectorization of text

In [19]:
vectorizer = TfidfVectorizer()
tf_idf_matrix_train = vectorizer.fit_transform(train_corpus)
tf_idf_matrix_test=vectorizer.transform(test_corpus)

Using keras to build a neural network architecutre

In [20]:
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical


In [21]:
y_train = to_categorical(train['QType-Coarse'],
train['QType-Coarse'].nunique())
y_test = to_categorical(test['QType-Coarse'],
train['QType-Coarse'].nunique())


In [22]:
model=Sequential()
model.add(Dense(128,activation='relu',input_dim=tf_idf_matrix_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(6,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1027968   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 1028742 (3.92 MB)
Trainable params: 1028742 (3.92 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


TensorFlow expects the indices in your sparse matrix to be sorted. This is a common requirement for operations on sparse matrices in TensorFlow.

To resolve this issue, you can reorder the indices of your sparse matrix before passing it to the model. We can use the tf.sparse.reorder function to create a correctly ordered copy of your sparse tensor.

If tf_idf_matrix_train is not already a tf.SparseTensor, We’ll need to convert it first.

In [28]:
from scipy.sparse import coo_matrix
import tensorflow as tf

# Convert Scipy sparse matrix to COO format
coo = coo_matrix(tf_idf_matrix_train)
coo2 = coo_matrix(tf_idf_matrix_test)
# Create TensorFlow SparseTensor from COO
tf_idf_matrix_train_tensor = tf.SparseTensor(indices=np.array([coo.row, coo.col]).T, values=coo.data, dense_shape=coo.shape)
tf_idf_matrix_test_tensor = tf.SparseTensor(indices=np.array([coo2.row, coo2.col]).T, values=coo2.data, dense_shape=coo2.shape)

# Reorder the SparseTensor
tf_idf_matrix_train_ordered = tf.sparse.reorder(tf_idf_matrix_train_tensor)
tf_idf_matrix_test_ordered = tf.sparse.reorder(tf_idf_matrix_test_tensor)

In [29]:
# import tensorflow as tf
# tf_idf_matrix_train_ordered = tf.sparse.reorder(tf_idf_matrix_train)

In [30]:
training_history = model.fit(tf_idf_matrix_train_ordered, y_train,epochs=10, batch_size=100)


Epoch 1/10
55/55 [==============================] - 2s 23ms/step - loss: 1.6995 - categorical_accuracy: 0.3601
Epoch 2/10
55/55 [==============================] - 1s 23ms/step - loss: 1.3471 - categorical_accuracy: 0.7241
Epoch 3/10
55/55 [==============================] - 1s 22ms/step - loss: 0.8879 - categorical_accuracy: 0.8850
Epoch 4/10
55/55 [==============================] - 1s 22ms/step - loss: 0.5510 - categorical_accuracy: 0.9312
Epoch 5/10
55/55 [==============================] - 1s 22ms/step - loss: 0.3602 - categorical_accuracy: 0.9534
Epoch 6/10
55/55 [==============================] - 1s 22ms/step - loss: 0.2467 - categorical_accuracy: 0.9714
Epoch 7/10
55/55 [==============================] - 1s 24ms/step - loss: 0.1774 - categorical_accuracy: 0.9813
Epoch 8/10
55/55 [==============================] - 1s 23ms/step - loss: 0.1349 - categorical_accuracy: 0.9851
Epoch 9/10
55/55 [==============================] - 1s 23ms/step - loss: 0.1013 - categorical_accuracy: 0.9908
E

In [32]:
loss,accuracy=model.evaluate(tf_idf_matrix_test_ordered,y_test,verbose=False)

In [33]:
print(f"Testing Accuracy:{accuracy}")

Testing Accuracy:0.8579999804496765


Saving model architecture and weight

In [35]:
model.save("model_name.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#TO download model from colab
from google.colab import files
files.download('model_name.h5')
